In [2]:
import os, os.path, re # ADDED, for dir operations
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

def numSamples(path):
    dirName = path
    # Get the list of all files in directory tree at given path
    listOfFiles = list()
    for (dirpath, dirnames, filenames) in os.walk(dirName):
        listOfFiles += [os.path.join(dirpath, file) for file in filenames]
        # Get the total count from the full list of all files in dir and subdir
        imgsInDir = [f for f in listOfFiles if re.search(r'.*\.(jpg)$', f)]
    return (len(imgsInDir)) # len() is required to return integer, else full list.

img_wh = 150 # 'global' variable
img_width, img_height = img_wh, img_wh # reduce modification to the original example
train_data_dir = './dogs-vs-cats/data1/train/' # should not require changing if dir setup properly
validation_data_dir = './dogs-vs-cats/data1/validation/' # should not require changing if dir setup properly
nb_train_samples = numSamples(train_data_dir) # ADDED
nb_validation_samples = numSamples(validation_data_dir) # ADDED
epochs = 10
batch_size = 10 # batch size determines 

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# convnet Block 1 layer 1
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# convnet Block 1 layer 2
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# convnet Block 1 layer 3
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Block 1 is connected to two fully-connected (fc) layers (flatten(fc1) and dense (fc2))
model.add(Flatten())
model.add(Dense(64)) 
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid')) 

#Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('draft_try.h5') # always save after or during training.


Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/10
200/200 [==============================] - 11s 57ms/step - loss: 0.7071 - acc: 0.5310 - val_loss: 0.6854 - val_acc: 0.5463
Epoch 2/10
200/200 [==============================] - 11s 54ms/step - loss: 0.6792 - acc: 0.5925 - val_loss: 0.6107 - val_acc: 0.6700
Epoch 3/10
200/200 [==============================] - 11s 56ms/step - loss: 0.6357 - acc: 0.6590 - val_loss: 0.6881 - val_acc: 0.6075
Epoch 4/10
200/200 [==============================] - 11s 56ms/step - loss: 0.6196 - acc: 0.6875 - val_loss: 0.5782 - val_acc: 0.7088
Epoch 5/10
200/200 [==============================] - 11s 57ms/step - loss: 0.5923 - acc: 0.6900 - val_loss: 0.5961 - val_acc: 0.6613
Epoch 6/10
200/200 [==============================] - 11s 57ms/step - loss: 0.5804 - acc: 0.7060 - val_loss: 0.5822 - val_acc: 0.6950
Epoch 7/10
200/200 [==============================] - 11s 56ms/step - loss: 0.5747 - acc: 0.7165 - val_loss: 0.53